In [12]:
import bs4 as beautifulsoup

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd

# -------- Setup Chrome Driver --------
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in background
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")

driver_path = r"E:\Program Files\chrome-win32\chromedriver.exe"  # Update path to your chromedriver
driver = webdriver.Chrome(service=Service(driver_path), options=chrome_options)

# -------- Categories --------
main_categories = {
    "Sports & Outdoors": "https://www.banggood.com/Wholesale-Sports-and-Outdoors-ca-6001.html",
    "Electronics": "https://www.banggood.com/Wholesale-Electronics-c-1001.html",
    "Home & Garden": "https://www.banggood.com/Wholesale-Home-Garden-c-1002.html",
    "Lights & Lighting": "https://www.banggood.com/Wholesale-Lights-Lighting-c-1003.html",
    "Computers & Accessories": "https://www.banggood.com/Wholesale-Computers-Video-Games-c-1004.html"
}

all_products = []

for cat_name, cat_url in main_categories.items():
    print(f"[MAIN CATEGORY] {cat_name}")
    driver.get(cat_url)
    time.sleep(3)  # Wait for page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # -------- Subcategories --------
    subcategories = []
    for li in soup.select("li.cate-item > a"):
        name = li.text.strip()
        url = li.get("href")
        subcategories.append((name, url))
    
    print(f"Found {len(subcategories)} subcategories")

    # -------- Scrape each subcategory --------
    for sub_name, sub_url in subcategories:
        print(f" Scraping subcategory: {sub_name}")
        driver.get(sub_url)
        time.sleep(3)
        
        products_scraped = 0
        page_num = 1
        
        while products_scraped < 20:  # Minimum 20 products per subcategory
            soup = BeautifulSoup(driver.page_source, "html.parser")
            product_list = soup.select("div.product-list ul li")
            if not product_list:
                print("  No products found on this page.")
                break
            
            for product in product_list:
                try:
                    name = product.select_one("span.img a img")["alt"].strip()
                    url = product.select_one("span.img a")["href"]
                    price_tag = product.select_one("span.price-box span")
                    price = price_tag.text.strip() if price_tag else ""
                    rating_tag = product.select_one("div.score-star span")
                    rating = rating_tag["style"].strip().replace("width:", "").replace("%", "") if rating_tag else ""
                    reviews_tag = product.select_one("span.bot-info span")
                    reviews = reviews_tag.text.strip() if reviews_tag else ""
                    
                    all_products.append({
                        "Category": cat_name,
                        "Subcategory": sub_name,
                        "Product Name": name,
                        "Price": price,
                        "Rating": rating,
                        "Reviews": reviews,
                        "URL": url
                    })
                    products_scraped += 1
                    if products_scraped >= 20:
                        break
                except Exception as e:
                    continue
            
            # -------- Next page --------
            page_num += 1
            next_page_url = f"{sub_url}?page={page_num}"
            driver.get(next_page_url)
            time.sleep(3)
            if len(product_list) == 0:
                break

# -------- Save to CSV --------
df = pd.DataFrame(all_products)
df.to_csv("banggood_products.csv", index=False)
print("Scraping completed. CSV saved as banggood_products.csv")

driver.quit()


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [6]:
print(len(all_data))


0


In [5]:
df.head()

""


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# -------- Categories --------

main_categories = {
"Sports & Outdoors": "[https://www.banggood.com/Wholesale-Sports-and-Outdoors-ca-6001.html](https://www.banggood.com/Wholesale-Sports-and-Outdoors-ca-6001.html)",
"Electronics": "[https://www.banggood.com/Wholesale-Electronics-c-1001.html](https://www.banggood.com/Wholesale-Electronics-c-1001.html)",
"Home & Garden": "[https://www.banggood.com/Wholesale-Home-Garden-c-1002.html](https://www.banggood.com/Wholesale-Home-Garden-c-1002.html)",
"Lights & Lighting": "[https://www.banggood.com/Wholesale-Lights-Lighting-c-1003.html](https://www.banggood.com/Wholesale-Lights-Lighting-c-1003.html)",
"Computers & Accessories": "[https://www.banggood.com/Wholesale-Computers-Video-Games-c-1004.html](https://www.banggood.com/Wholesale-Computers-Video-Games-c-1004.html)"
}

all_products = []

headers = {
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36"
}

for cat_name, cat_url in main_categories.items():
print(f"[MAIN CATEGORY] {cat_name}")
response = requests.get(cat_url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

```
# -------- Get subcategories --------
subcategories = []
for li in soup.select("li.cate-item > a"):
    name = li.text.strip()
    url = li.get("href")
    if url:
        subcategories.append((name, url))
print(f"Found {len(subcategories)} subcategories")

# -------- Scrape products in each subcategory --------
for sub_name, sub_url in subcategories:
    print(f" Scraping subcategory: {sub_name}")
    products_scraped = 0
    page_num = 1

    while products_scraped < 20:  # At least 20 products per subcategory
        page_response = requests.get(f"{sub_url}?page={page_num}", headers=headers)
        page_soup = BeautifulSoup(page_response.text, "html.parser")
        product_list = page_soup.select("div.product-list ul li")

        if not product_list:
            print("  No products found on this page.")
            break

        for product in product_list:
            try:
                name_tag = product.select_one("span.img a img")
                name = name_tag["alt"].strip() if name_tag else ""
                url_tag = product.select_one("span.img a")
                url = url_tag["href"] if url_tag else ""
                price_tag = product.select_one("span.price-box span")
                price = price_tag.text.strip() if price_tag else ""
                rating_tag = product.select_one("div.score-star span")
                rating = rating_tag["style"].strip().replace("width:", "").replace("%", "") if rating_tag else ""
                reviews_tag = product.select_one("span.bot-info span")
                reviews = reviews_tag.text.strip() if reviews_tag else ""

                all_products.append({
                    "Category": cat_name,
                    "Subcategory": sub_name,
                    "Product Name": name,
                    "Price": price,
                    "Rating": rating,
                    "Reviews": reviews,
                    "URL": url
                })
                products_scraped += 1
                if products_scraped >= 20:
                    break
            except Exception:
                continue

        page_num += 1
        time.sleep(1)
```

# -------- Save to CSV --------

df = pd.DataFrame(all_products)
df.to_csv("banggood_products.csv", index=False)
print("Scraping completed. CSV saved as banggood_products.csv")


IndentationError: expected an indented block after 'for' statement on line 22 (865697715.py, line 23)